In [1]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.2f}'.format
import seaborn as sns
import matplotlib.pyplot as plt
import hvplot as hvp
import datetime
import altair as alt
import seaborn as sns

### DataFrame for Drawdown + ROI

In [40]:
# Creating a range of dates
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2023, 12, 31)
date_range = pd.date_range(start_date, end_date, freq='D')
# Dates df
dates_df = pd.DataFrame(date_range, columns=['Date'])
#print(dates_df.head(1825))

# Creating a range of returns
portfolio_df = pd.DataFrame(
    {
    "fake1": np.random.uniform(1000, -500, 100),
    "fake2": np.random.uniform(1000, -500, 100),
    "fake3": np.random.uniform(1000, -500, 100)
    }
    )
f1_portfolio_df = portfolio_df["fake1"].to_frame()

# ROI df
ini_investment_df = pd.DataFrame(portfolio_df - np.random.rand(*portfolio_df.shape))
portfolio_df = portfolio_df.clip(lower=0)
unique_row_date = [portfolio_df.index[0] - pd.Timedelta(days=1)]

# Create a DataFrame for the unique row
unique_row_df = pd.DataFrame(, index=unique_row_date)

# Concatenate the unique row DataFrame with the original portfolio_df
# Make sure to sort the index after concatenation if necessary
portfolio_df = pd.concat([unique_row_df, portfolio_df]).sort_index()

# Find the minimum positive value in the DataFrame
min_positive_value = portfolio_df.min().min()
initial_investment_df = portfolio_df.applymap(lambda x: x - np.random.uniform(0, x))
initial_investment_df = initial_investment_df(column = {'fake1' :})

# Concatinating and finalizing test data
returns = pd.concat([dates_df, portfolio_df], join = "inner", axis = 1)
returns.set_index('Date', inplace = True)
returns = pd.DataFrame(returns)
returns = returns.reset_index().rename(columns={'Date': 'date'})
returns.set_index('date', inplace = True)
initial_investment_df.head()

,fake1,fake2,fake3
0,121.37,422.96,6.65
1,556.91,391.60,0.00
2,260.93,0.00,53.54
3,46.55,0.00,328.05
4,0.00,265.84,660.31


In [42]:
returns.head()

,fake1,fake2,fake3
date,,,
2018-01-01,588.46,723.83,118.73
2018-01-02,793.30,749.57,0.00
2018-01-03,480.28,0.00,676.21
2018-01-04,321.78,0.00,691.32
2018-01-05,0.00,346.79,673.65


### Data Frame for Portfolio Pie Chart

In [41]:
# Creating DataFrame for Bitcoin
btc_portfolio_df = pd.DataFrame({
    'stock': ['BTC:USD'],
    'position_size': np.random.uniform(10, 150, size = 1)
})

# Creating DataFrame for Stocks
stock_portfolio_df = pd.DataFrame({
    'stock': ['AMC', 'VSCO', 'TSLA', 'GOOGL'],
    'position_size': np.random.uniform(1000, 10000, 4)  # for 5 stocks
})

# ROI df
ini_investment_df = pd.DataFrame(portfolio_df - np.random.rand(*portfolio_df.shape))
portfolio_df = portfolio_df.clip(lower=0)

# Find the minimum positive value in the DataFrame
min_positive_value = portfolio_df.min().min()
initial_investment_df = portfolio_df.applymap(lambda x: x - np.random.uniform(0, x))
initial_investment_df = initial_investment_df(column = {'fake1' :})
# Combining DataFrames
size_portfolio_df = pd.concat([btc_portfolio_df, stock_portfolio_df], axis = 0).reset_index(drop = True)

# Renaming stock column to securities
size_portfolio_df = size_portfolio_df.rename(columns = {'stock' : 'securities'})



SyntaxError: invalid syntax (2793483973.py, line 20)

### Portfolio Pie Chart Function

In [4]:
# Defining Portfolio pie chart function
def portfolio_piechart (holdings, size):
    selection = alt.selection_single()
    pie_chart = alt.Chart(size_portfolio_df).mark_arc().encode(
        theta = alt.Theta(field = 'position_size', type = 'quantitative'),
        color = alt.Color(field = 'securities', type = 'nominal'),
        tooltip=['securities', 'position_size']
    )
    return pie_chart

### Portfolio Pie Chart

In [5]:
one = size_portfolio_df['securities']
two = size_portfolio_df['position_size']
user_in_holdings = portfolio_piechart(one, two)
user_in_holdings

c:\Users\Gino\AppData\Local\Programs\Python\Python39\lib\site-packages\altair\utils\deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


alt.Chart(...)

In [6]:
initial_investment = 10000

In [7]:
def cumulative_returns(df):
    return (1 + df).cumprod()

In [8]:
user_cr = cumulative_returns(portfolio_df)
user_cr

,fake1,fake2,fake3
0,549.87,732.14,789.40
1,45024.86,507906.29,329010.22
2,-14967482.38,258724617.19,-35488578.59
3,-12406545152.63,47467716618.41,-35026394642.85
4,4475917805392.61,39477795819965.52,3867069364669.15
...,...,...,...
95,22035003672701331096395754999335905465895408971...,37314922549350880971640162584610574353556892182...,98352860255176750136009519934792358324997028212...
96,-9818220492994853555487275163843993324670717272...,29517136477590520057082235319641635455331219196...,-4713479085088262482853842282509745952603715433...
97,-6379777459199688852721152639090297459912959222...,-6709653938833715933405501827399982514088501722...,-3607329611668087457782929878407645516423240977...
98,-1650496537538625770734877292319820586703586498...,13974556616424987175800567074585905774965368209...,-2970799102272079147620074667173492448057372935...


### Correlation

In [9]:
returns_cor = returns.corr()
returns_cor

,fake1,fake2,fake3
fake1,1.00,-0.05,-0.06
fake2,-0.05,1.00,-0.09
fake3,-0.06,-0.09,1.00


### Correlation Plot Function

In [46]:
# Seaborn Heatmap
# def correlation_chart(returns):
#     returns_cor = returns.corr()
#     corr_chart= sns.heatmap(returns_cor, vmin=-1 , vmax = 1)
#     return corr_chart

# Scatter plot function
def correlation_chart(returns_df, corr_df):
    returns_cor = returns.corr()

    # Brush for selection
    brush = alt.selection_interval()

    # Scatter Plot
    correlation_points = alt.Chart(returns).mark_point().encode(
        x = returns + ':Q',
        y = returns_cor + ':Q', 
        color=alt.condition(brush, alt.value('steelblue'), alt.value('grey'))
    ).add_params(brush)
    return correlation_points

    # Base chart for data tables
    ranked_text = alt.Chart(source).mark_text(align='right').encode(
        y=alt.Y('row_number:O').axis(None)
    ).transform_filter(
        brush
    ).transform_window(
        row_number='row_number()'
    ).transform_filter(
        alt.datum.row_number < 15
    )

    # Data Tables
    horsepower = ranked_text.encode(text='Horsepower:N').properties(
        title=alt.Title(text='Horsepower', align='right')
    )
    mpg = ranked_text.encode(text='Miles_per_Gallon:N').properties(
        title=alt.Title(text='MPG', align='right')
    )
    #origin = ranked_text.encode(text='Origin:N').properties(
        #title=alt.Title(text='Origin', align='right')
    #)
    text = alt.hconcat(horsepower, mpg, origin) # Combine data tables

    # Build chart
    alt.hconcat(
        points,
        text
    ).resolve_legend(
        color="independent"
    ).configure_view(
        stroke=None
    )

### Correlation Chart

In [47]:
# Seaborn plot
# correlation_chart(returns)
# correlation_chart
correlation_chart(returns, returns_cor)

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U2')) -> None

### Covariance

In [12]:
def covariance(df, ticker, market):
    return df[ticker].cov(df[market])

In [13]:
user_covariance = covariance(portfolio_df, 'fake2', 'fake3')
user_covariance

-16083.528151311688

### Variance

In [14]:
def variance(df, market):
    return df[market].var()

In [15]:
user_variance = variance(portfolio_df, 'fake3')
user_variance

206148.8193420841

### Drawdown

In [16]:
def drawdown(df, tickers):
    Roll_Max = df[tickers].cummax()
    Daily_Drawdown = df[tickers]/Roll_Max - 1.0
    Max_Daily_Drawdown = Daily_Drawdown.cummin() * 100

In [17]:
user_drawdown = drawdown(portfolio_df, 'fake1')
user_drawdown

### Drawdown Chart Function

In [18]:
def drawdown_chart(df, date, quant):
    '''
    df: dataframe
    date: str: should be a date column name
    quant: str: should be a quantitative data type
    '''
    chart = alt.Chart(df.reset_index()).mark_bar().encode(
        x = date + ':T',  # T specifies temporal data type for x-axis
        y = quant + ':Q',  # Q specifies a quantitative data type for y-axis
        color = alt.condition(
            getattr(alt.datum, quant) > 0,
            alt.value("Green"),  # The positive color
            alt.value("red")
    ), tooltip= [date + ':T', quant + ':Q']
    ).properties(width=800)
    chart = chart.interactive()
    return chart

In [19]:
user_drawdown = drawdown_chart(returns, 'date', 'fake1')
user_drawdown

alt.Chart(...)

In [20]:
# Line chart
def tracking_error(df, tickers, market):
    track_error = np.sqrt(sum([i**2 for i in df[tickers] - df[market]]))
    return track_error

In [21]:
user_tracking_error = tracking_error(portfolio_df, 'fake2', 'fake3')
user_tracking_error

6329.537269817434

### Beta

In [22]:

def beta(covariance, variance):
    chart_beta = covariance / variance
    chart_beta = chart_beta
    return chart_beta

In [23]:
user_beta = beta(user_covariance, user_variance)
user_beta

-0.0780190165660014

### Beta chart function

In [24]:
# In the works
# def beta_chart(covariance, variance):
#     chart_beta = covariance / variance
#     chart_beta = chart_beta.rolling(window = 21).plot()
#     plt.show()
#     beta.shape
#     chart_beta.plot()
#     return chart_beta

### Sharpe Ratio

In [25]:
def sharpe_ratio(df):
    sharpe = (df.mean()*252) / (df.std() * np.sqrt(252))
    return sharpe

In [26]:
user_sharpe_ratio = sharpe_ratio(portfolio_df)
user_sharpe_ratio

fake1   7.30
fake2   8.10
fake3   8.40
dtype: float64

### Return on Investment

In [27]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

### Return on Investment

In [28]:
def return_on_investment(investment, returns):
    cumulative_profit = investment * returns
    return_oi = (cumulative_profit - investment) / investment
    return(return_oi)

In [29]:
user_roi = return_on_investment(initial_investment, user_cr)
user_roi

,fake1,fake2,fake3
0,548.87,731.14,788.40
1,45023.86,507905.29,329009.22
2,-14967483.38,258724616.19,-35488579.59
3,-12406545153.63,47467716617.41,-35026394643.85
4,4475917805391.61,39477795819964.52,3867069364668.15
...,...,...,...
95,22035003672701333854221886550329554830259512686...,37314922549350880971640162584610574353556892182...,98352860255176750136009519934792358324997028212...
96,-9818220492994853555487275163843993324670717272...,29517136477590520057082235319641635455331219196...,-4713479085088262482853842282509745952603715433...
97,-6379777459199688852721152639090297459912959222...,-6709653938833715933405501827399982514088501722...,-3607329611668087457782929878407645516423240977...
98,-1650496537538625770734877292319820586703586498...,13974556616424987175800567074585905774965368209...,-2970799102272079147620074667173492448057372935...


### ROI Chart

In [30]:

def roi_chart(df, date, roi, initial_investment):
    roi_chart = alt.Chart(df).mark_bar().encode(
        x='project',
        y='score'
    ).properties(
        width=alt.Step(40)  # controls width of bar.
    )

    tick = alt.Chart(df).mark_tick(
        color='red',
        thickness=2,
        size=40 * 0.9,  # controls width of tick.
    ).encode(
        x = asset,
        y = roi
    )

    bar + tick